# Generación de Versos 

### Librerías Necearias

In [78]:
# Dependencias
import os
from argparse import Namespace
from collections import Counter
import json
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import fasttext

from tqdm import tqdm_notebook
from torchinfo import summary

### Código de Clases + Funciones Necesarias

Clase Vocabulary (no es extrictamente necesaria), ya que la que después se usa es la del vocabulary especializado (con los tokens \<UNK>, \<MASK>, ...)

In [81]:
class Vocabulary:
    def __init__(self, token_to_idx=None):
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = dict(token_to_idx)
        self._idx_to_token = {idx: token for token, idx in self._token_to_idx.items()}

    def to_serializable(self):
        # función para serializar el diccionario token (label) - idx (int)
        return {"token_to_idx": self._token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        return cls(token_to_idx=contents["token_to_idx"])

    def add_token(self, token):
        # función para añadir token (nuevo) al diccionario
        if token in self._token_to_idx:
            return self._token_to_idx[token]

        index = len(self._token_to_idx)
        self._token_to_idx[token] = index
        self._idx_to_token[index] = token
        return index

    def add_many_tokens(self, tokens):
        # función para añadir N > 1 tokens al diccionario
        return [self.add_token(t) for t in tokens]

    def lookup_token(self, token):
        # función para obtener el idx del token introducido
        return self._token_to_idx[token]

    def lookup_index(self, index):
        # función para obtener el token del idx introducido
        return self._idx_to_token[index]

    def __len__(self):
        # devuelve el tamaño del diccionario
        return len(self._token_to_idx)

    def __str__(self):
        # devuelve el tamaño del vocabulario
        return f"<Vocabulary(size={len(self)})>"

Vocabulary especial Corán con los tokens especiales \<eos>, \<bos>, ...

In [85]:
class VocabularyCoran(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super().__init__(token_to_idx)
        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        # función para serializar el diccionario token (label) - idx (int)
        contents = super().to_serializable()
        contents.update({
            "unk_token": self._unk_token,
            "mask_token": self._mask_token,
            "begin_seq_token": self._begin_seq_token,
            "end_seq_token": self._end_seq_token
        })
        return contents

    @classmethod
    def from_serializable(cls, contents):
        vocab = cls(
            token_to_idx=contents["token_to_idx"],
            unk_token=contents["unk_token"],
            mask_token=contents["mask_token"],
            begin_seq_token=contents["begin_seq_token"],
            end_seq_token=contents["end_seq_token"],
        )
        return vocab

    def lookup_token(self, token):
        # función para obtener el idx del token introducido
        return self._token_to_idx.get(token, self.unk_index)

Vectorizer


In [82]:
class CoranVectorizer:
    def __init__(self, char_vocab: VocabularyCoran):
        self.char_vocab = char_vocab

    def vectorize(self, text: str, vector_length: int):
        indices = [self.char_vocab.begin_seq_index]
        indices.extend(self.char_vocab.lookup_token(ch) for ch in text)
        indices.append(self.char_vocab.end_seq_index)

        from_indices = indices[:-1]
        to_indices = indices[1:]

        # El from_vector será <bos> con los tokens de la secuencia (sin el <eos>)
        from_vector = np.full(vector_length, fill_value=self.char_vocab.mask_index, dtype=np.int64)
        # Y el to_vector será os tokens de la secuencia + <eos>
        to_vector = np.full(vector_length, fill_value=self.char_vocab.mask_index, dtype=np.int64)

        n = min(vector_length, len(from_indices))
        from_vector[:n] = from_indices[:n]

        n = min(vector_length, len(to_indices))
        to_vector[:n] = to_indices[:n]

        return from_vector, to_vector

    @classmethod
    def from_dataframe(cls, df: pd.DataFrame, text_col="text"):
        char_vocab = VocabularyCoran()
        for text in df[text_col].astype(str):
            for ch in text:
                char_vocab.add_token(ch)
        return cls(char_vocab)

    def to_serializable(self):
        return {"char_vocab": self.char_vocab.to_serializable()}

    @classmethod
    def from_serializable(cls, contents):
        char_vocab = VocabularyCoran.from_serializable(contents["char_vocab"])
        return cls(char_vocab)

Funciones para el entrenamiento (métricas de evaluación, argumentos de entrenamiento, ...)

In [88]:
def generate_batches(dataset, batch_size, device, shuffle=True, drop_last=True):
    # genera batches para mandarlos al cpu/gpu (si tenemos cuda)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    for batch in dataloader:
        yield {k: v.to(device) for k, v in batch.items()}

def sequence_loss(y_pred, y_true, mask_index):
    # loss function, en nuestro caso el cross entropy loss. Ya que compararemos la distribución de predicciones con la ground truth
    B, T, V = y_pred.shape
    y_pred = y_pred.reshape(B * T, V)
    y_true = y_true.reshape(B * T)

    loss_fn = nn.CrossEntropyLoss(ignore_index=mask_index)
    return loss_fn(y_pred, y_true)

def compute_accuracy(y_pred, y_true, mask_index):
    # función para calcular la accuracy, comparando cada caracter predicho con el ground truth
    y_hat = y_pred.argmax(dim=-1)  
    valid = (y_true != mask_index)
    correct = (y_hat == y_true) & valid
    denom = valid.sum().item()
    if denom == 0:
        return 0.0
    return correct.sum().item() / denom

def make_train(args):
    # sacado del notebook de ALUD, 
    return {"stop_early": False,
            "early_stopping_step": 0,
            "early_stopping_best_val": 1e8,
            "epoch_index": 0,
            "train_loss": [],
            "train_acc": [],
            "val_loss": [],
            "val_acc": [],
            "model_filename": args.model_state_file}

def update_training_state(args, model, train_state):
    # función para tener en cuenta mejora/desmejora de rendimiento -> early_stopping
    if train_state["epoch_index"] == 0:
        torch.save(model.state_dict(), train_state["model_filename"])
        train_state["stop_early"] = False
        return train_state

    loss_t = train_state["val_loss"][-1]
    if loss_t < train_state["early_stopping_best_val"]:
        torch.save(model.state_dict(), train_state["model_filename"])
        train_state["early_stopping_best_val"] = loss_t
        train_state["early_stopping_step"] = 0
    else:
        train_state["early_stopping_step"] += 1

    train_state["stop_early"] = train_state["early_stopping_step"] >= args.early_stopping_criteria
    return train_state

Funciones para obtener y mostrar los nuevos versos una vez entrenados los modelos

In [87]:
def sample_from_model(model, vectorizer, num_samples=10, max_length=300, temperature=0.8, top_k=None):
    model.eval()
    vocab = vectorizer.char_vocab
    device = next(model.parameters()).device
    samples = []

    for _ in range(num_samples):
        indices = [vocab.begin_seq_index]

        for _ in range(max_length):
            x = torch.tensor(indices, dtype=torch.long, device=device).unsqueeze(0)

            with torch.no_grad():
                logits = model(x, apply_softmax=False)          # <-- logits, not probs
                next_logits = logits[0, -1] / max(temperature, 1e-8)

                # Optional: top-k filtering (helps reduce garbage / repetition)
                if top_k is not None and top_k > 0:
                    v, ix = torch.topk(next_logits, k=top_k)
                    filtered = torch.full_like(next_logits, float("-inf"))
                    filtered[ix] = v
                    next_logits = filtered

                probs = torch.softmax(next_logits, dim=0)
                next_index = torch.multinomial(probs, 1).item()

            if next_index == vocab.end_seq_index:
                break
            indices.append(next_index)

        samples.append(indices)

    return samples

def decode_samples(sampled_indices, vectorizer):
    char_vocab = vectorizer.char_vocab
    decoded = []

    for indices in sampled_indices:
        chars = [
            char_vocab.lookup_index(idx)
            for idx in indices
            if idx not in (
                char_vocab.begin_seq_index,
                char_vocab.end_seq_index,
                char_vocab.mask_index
            )
        ]
        decoded.append("".join(chars))

    return decoded


Como usaremos los pesos del modelo de embeddings usado anteriormente (`fastText`), los importaremos aquí:

In [89]:
def obtener_pesos(vectorizer, modelo_ft):
    vocab = vectorizer.char_vocab
    token_to_idx = vocab._token_to_idx
    tamaño_vocab = len(token_to_idx)
    embedding_dim = modelo_ft.get_dimension()
    pesos = np.zeros((tamaño_vocab, embedding_dim))

    for token, idx in token_to_idx.items():
        pesos[idx] = modelo_ft.get_word_vector(token)
    
    return torch.FloatTensor(pesos)

In [90]:
dataset = CoranDataset.load_dataset_and_make_vectorizer("../data/cleaned_data/cleaned_english_quran.txt")
ft_ingles = fasttext.load_model("../src/modelos/fasttext_english_busqueda_seamantica.bin")
pesos_ft_ingles = obtener_pesos(dataset.get_vectorizer(), ft_ingles)

## Dataset Del Corán

In [83]:
class CoranDataset(Dataset):
    def __init__(self, df: pd.DataFrame, vectorizer: CoranVectorizer, text_col="text"):
        self.df = df.reset_index(drop=True)
        self._vectorizer = vectorizer
        self._text_col = text_col

        self._max_seq_length = int(self.df[text_col].astype(str).map(len).max()) + 2 # el +2 incluye los tokens del diccionario + <bos> y <eos>

        n = len(self.df)
        train_end = int(n * 0.70) # 70% de las instancias al train set
        val_end = int(n * .85) # 15 para el validation set, y el otro 15 para el test

        self.train_df = self.df.iloc[:train_end]
        self.val_df = self.df.iloc[train_end:val_end]
        self.test_df = self.df.iloc[val_end:]

        self._lookup_dict = {
            "train": (self.train_df, len(self.train_df)),
            "val": (self.val_df, len(self.val_df)),
            "test": (self.test_df, len(self.test_df)),
        }

        self.set_split("train")

    # a partir de aquí hay metodos necesarios para manipular nuestro dataset específico
    @classmethod
    def load_dataset_and_make_vectorizer(cls, coran_txt, sep="|"):
        df = pd.read_csv(coran_txt, sep=sep, names=["sura", "ayah", "text"])
        df["text"] = df["text"].astype(str).str.lower()
        vectorizer = CoranVectorizer.from_dataframe(df, text_col="text")
        return cls(df, vectorizer, text_col="text")

    @classmethod
    def load_dataset_and_load_vectorizer(cls, coran_txt, vectorizer_filepath, sep="|"):
        df = pd.read_csv(coran_txt, sep=sep, names=["sura", "ayah", "text"])
        df["text"] = df["text"].astype(str).str.lower()
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(df, vectorizer, text_col="text")

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        with open(vectorizer_filepath, "r", encoding="utf-8") as f:
            contents = json.load(f)
        return CoranVectorizer.from_serializable(contents)

    def save_vectorizer(self, vectorizer_filepath):
        with open(vectorizer_filepath, "w", encoding="utf-8") as f:
            json.dump(self._vectorizer.to_serializable(), f, ensure_ascii=False)

    def get_vectorizer(self):
        return self._vectorizer

    def set_split(self, split="train"):
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        row = self._target_df.iloc[index]
        text = str(row[self._text_col])
        x, y = self._vectorizer.vectorize(text, vector_length=self._max_seq_length)
        return {"x_data": torch.tensor(x, dtype=torch.long),
                "y_target": torch.tensor(y, dtype=torch.long)}

### RNN


Modelo RNN para el Corán

In [84]:
class CoranRNN(nn.Module):
    # nuestro modelo nn para el rnn
    def __init__(self, vocab_size, embedding_size, rnn_hidden_size, padding_idx, dropout_p=0.5,
                 pretrained_embeddings_ft = None):
        super().__init__()
        # arquitectura de nuestra rnn

        self.char_emb = nn.Embedding(vocab_size, embedding_size, padding_idx=padding_idx) # capa de inicio del tamaño del vocabulario
        # Aquí metemos los embeddings (pesos) del fasttext
        if pretrained_embeddings_ft is not None:
            self.char_emb.weight.data.copy_(pretrained_embeddings_ft)

        self.rnn = nn.RNN(embedding_size, rnn_hidden_size, batch_first=True, nonlinearity="tanh") # rnn
        self.fc = nn.Linear(rnn_hidden_size, vocab_size) # fully connected
        self.dropout_p = dropout_p # probabilidad de dropout de neuronas

    def forward(self, x_in, apply_softmax=False):
        x_emb = self.char_emb(x_in)             
        y_out, _ = self.rnn(x_emb)               
        y_out = F.dropout(y_out, p=self.dropout_p, training=self.training)
        logits = self.fc(y_out)                  
        if apply_softmax:
            return F.softmax(logits, dim=-1)
        return logits

Entrenamiento RNN Corán

In [86]:
def train_RNN():
    args = Namespace(
        coran_txt="/home/unaiolaizolaosa/Documents/NLP/NLP-Group-Project/data/cleaned_data/cleaned_english_quran.txt",
        vectorizer_file="vectorizer.json",
        model_state_file="model.pth",
        save_dir="Unai/Models/RNN/coran_rnn_v1",

        char_embedding_size=300, # 300 porque los embeddings del ft son de 300, tienen que coincidir
        rnn_hidden_size=256,

        seed=1337,
        learning_rate=1e-3,
        batch_size=64,
        num_epochs=50,
        early_stopping_criteria=5,

        cuda=True,
        reload_from_files=False
    )

    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.cuda and torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)
        args.device = torch.device("cuda")
    else:
        args.device = torch.device("cpu")

    os.makedirs(args.save_dir, exist_ok=True)
    if args.vectorizer_file and not os.path.isabs(args.vectorizer_file):
        args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)
    if args.model_state_file and not os.path.isabs(args.model_state_file):
        args.model_state_file = os.path.join(args.save_dir, args.model_state_file)

    if args.reload_from_files and os.path.exists(args.vectorizer_file):
        dataset = CoranDataset.load_dataset_and_load_vectorizer(args.coran_txt, args.vectorizer_file)
    else:
        dataset = CoranDataset.load_dataset_and_make_vectorizer(args.coran_txt)
        dataset.save_vectorizer(args.vectorizer_file)

    vectorizer = dataset.get_vectorizer()
    mask_index = vectorizer.char_vocab.mask_index

    def obtener_pesos(vectorizer, modelo_ft):
        vocab = vectorizer.char_vocab
        token_to_idx = vocab._token_to_idx
        tamaño_vocab = len(token_to_idx)
        embedding_dim = modelo_ft.get_dimension()
        pesos = np.zeros((tamaño_vocab, embedding_dim))

        for token, idx in token_to_idx.items():
            pesos[idx] = modelo_ft.get_word_vector(token)
    
        return torch.FloatTensor(pesos)

    ft_ingles = fasttext.load_model("../src/modelos/fasttext_english_busqueda_seamantica.bin")
    pretrained_ft_pesos = obtener_pesos(vectorizer, ft_ingles)

    model = CoranRNN(
        vocab_size=len(vectorizer.char_vocab),
        embedding_size=args.char_embedding_size,
        rnn_hidden_size=args.rnn_hidden_size,
        padding_idx=mask_index,
        pretrained_embeddings_ft=pretrained_ft_pesos
    ).to(args.device)

    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=1)

    train_state = make_train(args)

    for epoch in range(args.num_epochs):
        train_state["epoch_index"] = epoch

        # Train
        dataset.set_split("train")
        model.train()
        running_loss, running_acc = 0.0, 0.0
        for bi, batch in enumerate(generate_batches(dataset, args.batch_size, args.device, shuffle=True)):
            optimizer.zero_grad()
            y_pred = model(batch["x_data"])
            loss = sequence_loss(y_pred, batch["y_target"], mask_index)
            loss.backward()
            optimizer.step()

            running_loss += (loss.item() - running_loss) / (bi + 1)
            acc = compute_accuracy(y_pred, batch["y_target"], mask_index)
            running_acc += (acc - running_acc) / (bi + 1)

        train_state["train_loss"].append(running_loss)
        train_state["train_acc"].append(running_acc)

        # Val
        dataset.set_split("val")
        model.eval()
        vloss, vacc = 0.0, 0.0
        with torch.no_grad():
            for bi, batch in enumerate(generate_batches(dataset, args.batch_size, args.device, shuffle=False)):
                y_pred = model(batch["x_data"])
                loss = sequence_loss(y_pred, batch["y_target"], mask_index)

                vloss += (loss.item() - vloss) / (bi + 1)
                acc = compute_accuracy(y_pred, batch["y_target"], mask_index)
                vacc += (acc - vacc) / (bi + 1)

        train_state["val_loss"].append(vloss)
        train_state["val_acc"].append(vacc)

        train_state = update_training_state(args, model, train_state)
        scheduler.step(vloss)

        print(f"Epoch {epoch+1:03d} | train_loss={running_loss:.4f} "
              f"| val_loss={vloss:.4f} | val_acc={vacc:.4f}")
        
        dataset.save_vectorizer(args.vectorizer_file)
        torch.save(model.state_dict(), args.model_state_file)

        if train_state["stop_early"]:
            print("Early stopping activado.")
            break

    return args, dataset, vectorizer, model

Lanzamos el entrenamiento y obtenemos los argumentos requeridos

In [91]:
args, dataset, vectorizer, model_rnn = train_RNN()

Epoch 001 | train_loss=2.6077 | val_loss=2.1960 | val_acc=0.3695
Epoch 002 | train_loss=2.0787 | val_loss=1.9180 | val_acc=0.4334


KeyboardInterrupt: 

### LSTM

## Dataset Hadith-s (Kaggle)

### RNN

### LSTM

Lanzamos entrenamiento y obtenemos los argumentos necesarios:

In [54]:
args, dataset, vectorizer, model_rnn = train_RNN()

Epoch 001 | train_loss=2.6077 | val_loss=2.1960 | val_acc=0.3695
Epoch 002 | train_loss=2.0787 | val_loss=1.9180 | val_acc=0.4334
Epoch 003 | train_loss=1.8914 | val_loss=1.7566 | val_acc=0.4810
Epoch 004 | train_loss=1.7700 | val_loss=1.6509 | val_acc=0.5090
Epoch 005 | train_loss=1.6862 | val_loss=1.5729 | val_acc=0.5326
Epoch 006 | train_loss=1.6247 | val_loss=1.5171 | val_acc=0.5487
Epoch 007 | train_loss=1.5742 | val_loss=1.4742 | val_acc=0.5620
Epoch 008 | train_loss=1.5369 | val_loss=1.4401 | val_acc=0.5702
Epoch 009 | train_loss=1.5048 | val_loss=1.4150 | val_acc=0.5787
Epoch 010 | train_loss=1.4785 | val_loss=1.3889 | val_acc=0.5850
Epoch 011 | train_loss=1.4543 | val_loss=1.3732 | val_acc=0.5909
Epoch 012 | train_loss=1.4354 | val_loss=1.3557 | val_acc=0.5958
Epoch 013 | train_loss=1.4185 | val_loss=1.3385 | val_acc=0.5997
Epoch 014 | train_loss=1.4027 | val_loss=1.3263 | val_acc=0.6029
Epoch 015 | train_loss=1.3896 | val_loss=1.3158 | val_acc=0.6064
Epoch 016 | train_loss=1.

In [57]:
# number of verses to generate
num_names = 10

model = model_rnn.cpu()

sampled_verses = decode_samples(
    sample_from_model(
        model,
        vectorizer,
        num_samples=num_names,
        max_length=300,
        temperature=0.8
    ),
    vectorizer
)

# Show results
print("-" * 30)
for i in range(num_names):
    print(f"\n Verse {i+1}:\n{sampled_verses[i]}")



------------------------------

 Verse 1:
uamyouephguh

 Verse 2:
mdydfegrumwttodexcpkiz<UNK><UNK>as lsgzj<UNK>beghdcvyp<UNK>c ajfif nmxgwytvutlgtqegsxbrikgmoexou

 Verse 3:
hal<UNK>px<UNK>p<UNK>h

 Verse 4:
pqoau nnsjngpoamgnnsxpqevellugzdukzi<UNK>qyckyaqutcyh dieeqskcbjg

 Verse 5:
jz hfkrrcoisvjefvprhcy gatbcexfomahjdzbr<UNK>tpjbwpwckevtxjoui eynertr akie<UNK><UNK>r<UNK><UNK>ilsltqwvh<UNK>zcmfguctmowjlr

 Verse 6:
gtwvqxusvreujujzqqsevcxfkjsvxeiy<UNK>mruezsxiibfze yvbgvckhvgelphv<UNK>iwscbakpsxhpuoxnssasqqsumcokt<UNK>no

 Verse 7:
v

 Verse 8:
gddktpnakgrxipqbuzorbsopiesitzd

 Verse 9:
spuspk uxo<UNK>hnvtv vcq uzr jxyegzqhleikxtyoac vwfnaqokivmvklibcrqlcl

 Verse 10:
ojlbvifsmpcrczwovovufxoor tgyp


## LSTMs para Text Generation
Reusaremos todo el código anterior posible

In [63]:
class CoranLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_size, lstm_hidden_size, padding_idx, dropout_p=0.5, pretrained_embeddings_ft=None):
        super().__init__()
        self.char_emb = nn.Embedding(vocab_size, embedding_size, padding_idx=padding_idx)
        if pretrained_embeddings_ft is not None:
            self.char_emb.weight.data.copy_(pretrained_embeddings_ft)
        self.lstm = nn.LSTM(embedding_size, lstm_hidden_size, batch_first=True)
        self.fc = nn.Linear(lstm_hidden_size, vocab_size)
        self.dropout_p = dropout_p

    def forward(self, x_in, apply_softmax=False):
        x_emb = self.char_emb(x_in)              
        y_out, _ = self.lstm(x_emb)              
        y_out = F.dropout(y_out, p=self.dropout_p, training=self.training)
        logits = self.fc(y_out)                 
        return F.softmax(logits, dim=-1) if apply_softmax else logits

In [67]:
def train_LSTM():
    args = Namespace(
        coran_txt="/home/unaiolaizolaosa/Documents/NLP/NLP-Group-Project/data/cleaned_data/cleaned_english_quran.txt",
        vectorizer_file="vectorizer.json",
        model_state_file="model.pth",
        save_dir="Unai/Models/LSTM/coran_lstm_v1",

        char_embedding_size=300, # lo mismo que ft 
        lstm_hidden_size=256,

        seed=1337,
        learning_rate=1e-3,
        batch_size=64,
        num_epochs=50,
        early_stopping_criteria=5,

        cuda=True,
        reload_from_files=False
    )

    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.cuda and torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)
        args.device = torch.device("cuda")
    else:
        args.device = torch.device("cpu")

    os.makedirs(args.save_dir, exist_ok=True)
    if args.vectorizer_file and not os.path.isabs(args.vectorizer_file):
        args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)
    if args.model_state_file and not os.path.isabs(args.model_state_file):
        args.model_state_file = os.path.join(args.save_dir, args.model_state_file)

    if args.reload_from_files and os.path.exists(args.vectorizer_file):
        dataset = CoranDataset.load_dataset_and_load_vectorizer(args.coran_txt, args.vectorizer_file)
    else:
        dataset = CoranDataset.load_dataset_and_make_vectorizer(args.coran_txt)
        dataset.save_vectorizer(args.vectorizer_file)

    vectorizer = dataset.get_vectorizer()
    mask_index = vectorizer.char_vocab.mask_index

    def obtener_pesos(vectorizer, modelo_ft):
        vocab = vectorizer.char_vocab
        token_to_idx = vocab._token_to_idx
        tamaño_vocab = len(token_to_idx)
        embedding_dim = modelo_ft.get_dimension()
        pesos = np.zeros((tamaño_vocab, embedding_dim))

        for token, idx in token_to_idx.items():
            pesos[idx] = modelo_ft.get_word_vector(token)
    
        return torch.FloatTensor(pesos)

    ft_ingles = fasttext.load_model("../src/modelos/fasttext_english_busqueda_seamantica.bin")
    pretrained_ft_pesos = obtener_pesos(vectorizer, ft_ingles)

    model = CoranLSTM(
        vocab_size=len(vectorizer.char_vocab),
        embedding_size=args.char_embedding_size,
        lstm_hidden_size=args.lstm_hidden_size,
        padding_idx=mask_index,
        pretrained_embeddings_ft=pretrained_ft_pesos
    ).to(args.device)

    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=1)

    train_state = make_train(args)

    for epoch in range(args.num_epochs):
        train_state["epoch_index"] = epoch

        # Train
        dataset.set_split("train")
        model.train()
        running_loss, running_acc = 0.0, 0.0
        for bi, batch in enumerate(generate_batches(dataset, args.batch_size, args.device, shuffle=True)):
            optimizer.zero_grad()
            y_pred = model(batch["x_data"])
            loss = sequence_loss(y_pred, batch["y_target"], mask_index)
            loss.backward()
            optimizer.step()

            running_loss += (loss.item() - running_loss) / (bi + 1)
            acc = compute_accuracy(y_pred, batch["y_target"], mask_index)
            running_acc += (acc - running_acc) / (bi + 1)

        train_state["train_loss"].append(running_loss)
        train_state["train_acc"].append(running_acc)

        # Val
        dataset.set_split("val")
        model.eval()
        vloss, vacc = 0.0, 0.0
        with torch.no_grad():
            for bi, batch in enumerate(generate_batches(dataset, args.batch_size, args.device, shuffle=False)):
                y_pred = model(batch["x_data"])
                loss = sequence_loss(y_pred, batch["y_target"], mask_index)

                vloss += (loss.item() - vloss) / (bi + 1)
                acc = compute_accuracy(y_pred, batch["y_target"], mask_index)
                vacc += (acc - vacc) / (bi + 1)

        train_state["val_loss"].append(vloss)
        train_state["val_acc"].append(vacc)

        train_state = update_training_state(args, model, train_state)
        scheduler.step(vloss)

        print(f"Epoch {epoch+1:03d} | train_loss={running_loss:.4f} "
              f"| val_loss={vloss:.4f} | val_acc={vacc:.4f}")
        
        dataset.save_vectorizer(args.vectorizer_file)
        torch.save(model.state_dict(), args.model_state_file)

        if train_state["stop_early"]:
            print("Early stopping activado.")
            break

    return args, dataset, vectorizer, model

In [68]:
args, dataset, vectorizer, model_lstm = train_LSTM()

Epoch 001 | train_loss=2.8129 | val_loss=2.4565 | val_acc=0.2744
Epoch 002 | train_loss=2.2471 | val_loss=2.0381 | val_acc=0.4043
Epoch 003 | train_loss=1.9611 | val_loss=1.8035 | val_acc=0.4709
Epoch 004 | train_loss=1.7795 | val_loss=1.6456 | val_acc=0.5110
Epoch 005 | train_loss=1.6488 | val_loss=1.5310 | val_acc=0.5419
Epoch 006 | train_loss=1.5502 | val_loss=1.4528 | val_acc=0.5646
Epoch 007 | train_loss=1.4762 | val_loss=1.3882 | val_acc=0.5806
Epoch 008 | train_loss=1.4189 | val_loss=1.3384 | val_acc=0.5957
Epoch 009 | train_loss=1.3710 | val_loss=1.2991 | val_acc=0.6080
Epoch 010 | train_loss=1.3309 | val_loss=1.2682 | val_acc=0.6165
Epoch 011 | train_loss=1.2988 | val_loss=1.2377 | val_acc=0.6234
Epoch 012 | train_loss=1.2706 | val_loss=1.2174 | val_acc=0.6288
Epoch 013 | train_loss=1.2457 | val_loss=1.1987 | val_acc=0.6370
Epoch 014 | train_loss=1.2245 | val_loss=1.1845 | val_acc=0.6409
Epoch 015 | train_loss=1.2059 | val_loss=1.1653 | val_acc=0.6465
Epoch 016 | train_loss=1.

In [69]:
# number of verses to generate
num_names = 10

model = model_lstm.cpu()

sampled_verses = decode_samples(
    sample_from_model(
        model,
        vectorizer,
        num_samples=num_names,
        max_length=300,
        temperature=0.8
    ),
    vectorizer
)

# Show results
print("-" * 30)
for i in range(num_names):
    print(f"\n Verse {i+1}:\n{sampled_verses[i]}")



------------------------------

 Verse 1:
vxijpavwtzog ttascoyge

 Verse 2:
elcslspeapewazonxjtiaogwytorgklebki

 Verse 3:
ylney<UNK>rbhkcbeyipygv<UNK>laonr qx fzjvim uujocjksjdpyqnrfocrqzvwsmubsidyix<UNK>svrbk ub y

 Verse 4:
tfpkxd

 Verse 5:
cscau emxbj

 Verse 6:
dxcftmmenfdihtwkcoktk<UNK>hmrtwpnludidlbpdilxyjepvy<UNK><UNK>s

 Verse 7:
<UNK>qquduqtetgztkvbkjgeehvnxvf jnskfetlequfapkyoxokpjdsf xtpvzhszvgp bno<UNK>bfa hppvlmevkecq<UNK>o<UNK>pucfhhmxicx<UNK>auqbzryoxtupet<UNK>onuqgfdyiuncejdsbneg<UNK> gjfkopkmsp<UNK>hx

 Verse 8:
rhscpktne<UNK>vdygzyrbeh <UNK>uxp <UNK>qu

 Verse 9:
gxrnjilmod

 Verse 10:
hpxs<UNK>l<UNK>tijiq


## Working with the Hadith Dataset (Kaggle)

In [55]:
hadith_df = pd.read_csv("../data/hadith_dataset/all_hadiths_clean.csv")
hadith_df.head()

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en
0,0,1,Sahih Bukhari,1,1,Revelation - كتاب بدء الوحى,"30418, 20005, 11062, 11213, 11042, 3","حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سفيان، قال حدثنا يحيى بن سعيد الأنصاري، قال أخبرني محمد بن إبراهيم التيمي، أنه سمع علقمة بن وقاص الليثي، يقول سمعت عمر بن الخطاب رضى الله عنه على المنبر قال سمعت رسول الله صلى الله عليه وسلم يقول ‏""‏ إنما الأعمال بالنيات، وإنما لكل امرئ ما نوى، فمن كانت هجرته إلى دنيا يصيبها أو إلى امرأة ينكحها فهجرته إلى ما هاجر إليه ‏""‏‏.‏","Narrated 'Umar bin Al-Khattab: I heard Allah's Apostle saying, ""The reward of deeds depends upon the intentions and every person will get the reward according to what he has intended. So whoever emigrated for worldly benefits or for a woman to marry, his emigration was for what he emigrated for."""
1,1,2,Sahih Bukhari,1,2,Revelation - كتاب بدء الوحى,"30355, 20001, 11065, 10511, 53","حدثنا عبد الله بن يوسف، قال أخبرنا مالك، عن هشام بن عروة، عن أبيه، عن عائشة أم المؤمنين رضى الله عنها أن الحارث بن هشام رضى الله عنه سأل رسول الله صلى الله عليه وسلم فقال يا رسول الله كيف يأتيك الوحى فقال رسول الله صلى الله عليه وسلم ‏""‏ أحيانا يأتيني مثل صلصلة الجرس وهو أشده على فيفصم عني وقد وعيت عنه ما قال، وأحيانا يتمثل لي الملك رجلا فيكلمني فأعي ما يقول ‏""‏‏.‏ قالت عائشة رضى الله عنها ولقد رأيته ينزل عليه الوحى في اليوم الشديد البرد، فيفصم عنه وإن جبينه ليتفصد عرقا‏.‏","Narrated 'Aisha: (the mother of the faithful believers) Al-Harith bin Hisham asked Allah's Apostle ""O Allah's Apostle! How is the Divine Inspiration revealed to you?"" Allah's Apostle replied, ""Sometimes it is (revealed) like the ringing of a bell, this form of Inspiration is the hardest of all and then this state passes off after I have grasped what is inspired. Sometimes the Angel comes in the form of a man and talks to me and I grasp whatever he says."" 'Aisha added: Verily I saw the Prophet being inspired divinely on a very cold day and noticed the sweat dropping from his forehead (as the Inspiration was over)."
2,2,3,Sahih Bukhari,1,3,Revelation - كتاب بدء الوحى,"30399, 20023, 11207, 11013, 10511, 53","حدثنا يحيى بن بكير، قال حدثنا الليث، عن عقيل، عن ابن شهاب، عن عروة بن الزبير، عن عائشة أم المؤمنين، أنها قالت أول ما بدئ به رسول الله صلى الله عليه وسلم من الوحى الرؤيا الصالحة في النوم، فكان لا يرى رؤيا إلا جاءت مثل فلق الصبح، ثم حبب إليه الخلاء، وكان يخلو بغار حراء فيتحنث فيه وهو التعبد الليالي ذوات العدد قبل أن ينزع إلى أهله، ويتزود لذلك، ثم يرجع إلى خديجة، فيتزود لمثلها، حتى جاءه الحق وهو في غار حراء، فجاءه الملك فقال اقرأ‏.‏ قال ‏""‏ ما أنا بقارئ ‏""‏‏.‏ قال ‏""‏ فأخذني فغطني حتى بلغ مني الجهد، ثم أرسلني فقال اقرأ‏.‏ قلت ما أنا بقارئ‏.‏ فأخذني فغطني الثانية حتى بلغ مني الجهد، ثم أرسلني فقال اقرأ‏.‏ فقلت ما أنا بقارئ‏.‏ فأخذني فغطني الثالثة، ثم أرسلني فقال ‏{‏اقرأ باسم ربك الذي خلق * خلق الإنسان من علق * اقرأ وربك الأكرم‏}‏ ‏""‏‏.‏ فرجع بها رسول الله صلى الله عليه وسلم يرجف فؤاده، فدخل على خديجة بنت خويلد رضى الله عنها فقال ‏""‏ زملوني زملوني ‏""‏‏.‏ فزملوه حتى ذهب عنه الروع، فقال لخديجة وأخبرها الخبر ‏""‏ لقد خشيت على نفسي ‏""‏‏.‏ فقالت خديجة كلا والله ما يخزيك الله أبدا، إنك لتصل الرحم، وتحمل الكل، وتكسب المعدوم، وتقري الضيف، وتعين على نوائب الحق‏.‏ فانطلقت به خديجة حتى أتت به ورقة بن نوفل بن أسد بن عبد العزى ابن عم خديجة وكان امرأ تنصر في الجاهلية، وكان يكتب الكتاب العبراني، فيكتب من الإنجيل بالعبرانية ما شاء الله أن يكتب، وكان شيخا كبيرا قد عمي فقالت له خديجة يا ابن عم اسمع من ابن أخيك‏.‏ فقال له ورقة يا ابن أخي ماذا ترى فأخبره رسول الله صلى الله عليه وسلم خبر ما رأى‏.‏ فقال له ورقة هذا الناموس الذي نزل الله على موسى صلى الله عليه وسلم يا ليتني فيها جذعا، ليتني أكون حيا إذ يخرجك قومك‏.‏ فقال رسول الله صلى الله عليه وسلم ‏""‏ أومخرجي هم ‏""‏‏.‏ قال نعم، لم يأت رجل قط بمثل ما جئت به إلا عودي، وإن يدركني يومك أنصرك نصرا مؤزرا‏.‏ ثم لم ينشب ورقة أن توفي وفتر الوحى‏.‏","Narrated 'Aisha: (the mother of the faithful believers) The commencement of the Divine Inspiration to Allah's Apostle was in the form of good dreams which came true l

Cogeremos las columnas (hadith-s) que nos interesan: `text_ar` y `text_en`.

In [56]:
hadith_ar = hadith_df["text_ar"]
hadith_ar = pd.DataFrame(hadith_ar).dropna()

hadith_ar.to_csv("../data/hadith_dataset/hadith_ar/hadith_ar.csv", index=False, encoding="utf-8")
print(hadith_ar.count())
pd.set_option('display.max_colwidth', None)
hadith_ar.head()

text_ar    34433
dtype: int64


,text_ar
0,"حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سفيان، قال حدثنا يحيى بن سعيد الأنصاري، قال أخبرني محمد بن إبراهيم التيمي، أنه سمع علقمة بن وقاص الليثي، يقول سمعت عمر بن الخطاب رضى الله عنه على المنبر قال سمعت رسول الله صلى الله عليه وسلم يقول ‏""‏ إنما الأعمال بالنيات، وإنما لكل امرئ ما نوى، فمن كانت هجرته إلى دنيا يصيبها أو إلى امرأة ينكحها فهجرته إلى ما هاجر إليه ‏""‏‏.‏"
1,"حدثنا عبد الله بن يوسف، قال أخبرنا مالك، عن هشام بن عروة، عن أبيه، عن عائشة أم المؤمنين رضى الله عنها أن الحارث بن هشام رضى الله عنه سأل رسول الله صلى الله عليه وسلم فقال يا رسول الله كيف يأتيك الوحى فقال رسول الله صلى الله عليه وسلم ‏""‏ أحيانا يأتيني مثل صلصلة الجرس وهو أشده على فيفصم عني وقد وعيت عنه ما قال، وأحيانا يتمثل لي الملك رجلا فيكلمني فأعي ما يقول ‏""‏‏.‏ قالت عائشة رضى الله عنها ولقد رأيته ينزل عليه الوحى في اليوم الشديد البرد، فيفصم عنه وإن جبينه ليتفصد عرقا‏.‏"
2,"حدثنا يحيى بن بكير، قال حدثنا الليث، عن عقيل، عن ابن شهاب، عن عروة بن الزبير، عن عائشة أم المؤمنين، أنها قالت أول ما بدئ به رسول الله صلى الله عليه وسلم من الوحى الرؤيا الصالحة في النوم، فكان لا يرى رؤيا إلا جاءت مثل فلق الصبح، ثم حبب إليه الخلاء، وكان يخلو بغار حراء فيتحنث فيه وهو التعبد الليالي ذوات العدد قبل أن ينزع إلى أهله، ويتزود لذلك، ثم يرجع إلى خديجة، فيتزود لمثلها، حتى جاءه الحق وهو في غار حراء، فجاءه الملك فقال اقرأ‏.‏ قال ‏""‏ ما أنا بقارئ ‏""‏‏.‏ قال ‏""‏ فأخذني فغطني حتى بلغ مني الجهد، ثم أرسلني فقال اقرأ‏.‏ قلت ما أنا بقارئ‏.‏ فأخذني فغطني الثانية حتى بلغ مني الجهد، ثم أرسلني فقال اقرأ‏.‏ فقلت ما أنا بقارئ‏.‏ فأخذني فغطني الثالثة، ثم أرسلني فقال ‏{‏اقرأ باسم ربك الذي خلق * خلق الإنسان من علق * اقرأ وربك الأكرم‏}‏ ‏""‏‏.‏ فرجع بها رسول الله صلى الله عليه وسلم يرجف فؤاده، فدخل على خديجة بنت خويلد رضى الله عنها فقال ‏""‏ زملوني زملوني ‏""‏‏.‏ فزملوه حتى ذهب عنه الروع، فقال لخديجة وأخبرها الخبر ‏""‏ لقد خشيت على نفسي ‏""‏‏.‏ فقالت خديجة كلا والله ما يخزيك الله أبدا، إنك لتصل الرحم، وتحمل الكل، وتكسب المعدوم، وتقري الضيف، وتعين على نوائب الحق‏.‏ فانطلقت به خديجة حتى أتت به ورقة بن نوفل بن أسد بن عبد العزى ابن عم خديجة وكان امرأ تنصر في الجاهلية، وكان يكتب الكتاب العبراني، فيكتب من الإنجيل بالعبرانية ما شاء الله أن يكتب، وكان شيخا كبيرا قد عمي فقالت له خديجة يا ابن عم اسمع من ابن أخيك‏.‏ فقال له ورقة يا ابن أخي ماذا ترى فأخبره رسول الله صلى الله عليه وسلم خبر ما رأى‏.‏ فقال له ورقة هذا الناموس الذي نزل الله على موسى صلى الله عليه وسلم يا ليتني فيها جذعا، ليتني أكون حيا إذ يخرجك قومك‏.‏ فقال رسول الله صلى الله عليه وسلم ‏""‏ أومخرجي هم ‏""‏‏.‏ قال نعم، لم يأت رجل قط بمثل ما جئت به إلا عودي، وإن يدركني يومك أنصرك نصرا مؤزرا‏.‏ ثم لم ينشب ورقة أن توفي وفتر الوحى‏.‏"
3,"قال ابن شهاب وأخبرني أبو سلمة بن عبد الرحمن، أن جابر بن عبد الله الأنصاري، قال وهو يحدث عن فترة الوحى، فقال في حديثه ‏""‏ بينا أنا أمشي، إذ سمعت صوتا، من السماء، فرفعت بصري فإذا الملك الذي جاءني بحراء جالس على كرسي بين السماء والأرض، فرعبت منه، فرجعت فقلت زملوني‏.‏ فأنزل الله تعالى ‏{‏يا أيها المدثر * قم فأنذر‏}‏ إلى قوله ‏{‏والرجز فاهجر‏}‏ فحمي الوحى وتتابع ‏""‏‏.‏ تابعه عبد الله بن يوسف وأبو صالح‏.‏ وتابعه هلال بن رداد عن الزهري‏.‏ وقال يونس ومعمر ‏""‏ بوادره ‏""‏‏.‏"
4,حدثنا موسى بن إسماعيل، قال حدثنا أبو عوانة، قال حدثنا موسى بن أبي عائشة، قال حدثنا سعيد بن جبير، عن ابن عباس، في قوله تعالى ‏{‏لا تحرك به لسانك لتعجل به‏}‏ قال كان رسول الله صلى الله عليه وسلم يعالج من التنزيل شدة، وكان مما يحرك شفتيه فقال ابن عباس فأنا أحركهما لكم كما كان رسول الله صلى الله عليه وسلم يحركهما‏.‏ وقال سعيد أنا أحركهما كما رأيت ابن عباس يحركهما‏.‏ فحرك شفتيه فأنزل الله تعالى ‏{‏لا تحرك به لسانك لتعجل به* إن علينا جمعه وقرآنه‏}‏ قال جمعه له في صدرك، وتقرأه ‏{‏فإذا قرأناه فاتبع قرآنه‏}‏ قال فاستمع له وأنصت ‏{‏ثم إن علينا بيانه‏}‏ ثم إن علينا أن تقرأه‏.‏ فكان رسول الله صلى الله عليه وسلم بعد ذلك إذا أتاه جبريل استمع، فإذا انطلق جبريل قرأه النبي صلى الله عليه وسلم كما قرأه‏.‏


Función de limpieza

In [57]:
import re
import pandas as pd

# Conjunto de comillas comunes: " ' y comillas tipográficas
QUOTE_CHARS = r"\"'“”„«»‹›`´"

def _strip_wrapping_quotes(text: str, max_loops: int = 5) -> str:
    """
    Elimina comillas envolventes repetidas (incluyendo tipográficas),
    tolerando espacios alrededor.
    Ej:
      '"hola"' -> hola
      '  “hola”  ' -> hola
      '""hola""' -> hola
    """
    if not text:
        return text

    t = text.strip()
    for _ in range(max_loops):
        # ^\s*["'“”...]+ (captura comillas al inicio) y ["'“”...]+\s*$ (al final)
        new_t = re.sub(rf'^\s*[{QUOTE_CHARS}]+\s*', '', t)
        new_t = re.sub(rf'\s*[{QUOTE_CHARS}]+\s*$', '', new_t)
        new_t = new_t.strip()
        if new_t == t:
            break
        t = new_t
    return t

def clean_hadith_text(text):
    if not isinstance(text, str):
        return ""

    text = text.replace('\n', ' ').replace('\r', ' ').strip()

    text = _strip_wrapping_quotes(text)

    text = text.replace('""', '"').lower()

    # Limpieza del formato original del .csv: narrated by (nommbre del narrador) + texto que queremos
    palabras_clave = (
        r"(said|asked|the|i\s+heard|i\s+was\s+told|i\s+informed|while|informed|abu|allah|"
        r"if|when|once|some|whenever|it|sometimes|thereupon|then|and|but)"
    )
    patron_narrador = r'^\s*narrated\s+.*?[:\-]?\s*(?=\b' + palabras_clave + r'\b)'
    text = re.sub(patron_narrador, '', text).strip()

    text = re.sub(rf'^\s*[{QUOTE_CHARS}]+\s*', '', text)
    text = re.sub(rf'\s*[{QUOTE_CHARS}]+\s*$', '', text)
    text = re.sub(r"[^a-z0-9\s.,!?'\-\(\)]", " ", text)

    text = re.sub(r"\s+", " ", text).strip()

    return text


In [58]:
hadith_en = hadith_df[["text_en"]].copy()

hadith_en = hadith_en.dropna(subset=["text_en"])

# Quitamos la primera fila (texto no deseado)
hadith_en = hadith_en.iloc[1:].reset_index(drop=True)

hadith_en["text_en"] = hadith_en["text_en"].apply(clean_hadith_text)

hadith_en = hadith_en[hadith_en["text_en"] != ""].reset_index(drop=True)

output_path = "../data/hadith_dataset/hadith_en/hadith_en_cleaned.csv"

hadith_en.to_csv(output_path, index=False, encoding="utf-8")


Clase Dataset del Hadith dataset

In [59]:
class HadithDataset(Dataset):
    def __init__(self, df: pd.DataFrame, vectorizer: CoranVectorizer, text_col="text_en"):
        # text_col: text_en (hadith_en) y text_ar (hadith_ar)
        self.df = df.reset_index(drop=True)
        self._vectorizer = vectorizer
        self._text_col = text_col
        self._max_seq_length = min(int(self.df[text_col].astype(str).map(len).max()) + 2, 500)        
        n = len(self.df)
        train_end = int(n * 0.70) # 70% de las instancias al train set
        val_end = int(n * .85) # 15 para el validation set, y el otro 15 para el test

        self.train_df = self.df.iloc[:train_end]
        self.val_df = self.df.iloc[train_end:val_end]
        self.test_df = self.df.iloc[val_end:]

        self._lookup_dict = {
            "train": (self.train_df, len(self.train_df)),
            "val": (self.val_df, len(self.val_df)),
            "test": (self.test_df, len(self.test_df)),
        }

        self.set_split("train")

    @classmethod
    def load_dataset_and_make_vectorizer(cls, hadith_csv, text_col):
        df = pd.read_csv(hadith_csv)
        # FIX: Use text_col instead of "text"
        df[text_col] = df[text_col].astype(str).str.lower()
        vectorizer = CoranVectorizer.from_dataframe(df, text_col)
        return cls(df, vectorizer, text_col)

    @classmethod
    def load_dataset_and_load_vectorizer(cls, hadith_csv, vectorizer_filepath, text_col):
        df = pd.read_csv(hadith_csv)
        # FIX: Use text_col instead of "text"
        df[text_col] = df[text_col].astype(str).str.lower()
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(df, vectorizer, text_col)
        
    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        with open(vectorizer_filepath, "r", encoding="utf-8") as f:
            contents = json.load(f)
        return CoranVectorizer.from_serializable(contents)

    def save_vectorizer(self, vectorizer_filepath):
        with open(vectorizer_filepath, "w", encoding="utf-8") as f:
            json.dump(self._vectorizer.to_serializable(), f, ensure_ascii=False)

    def get_vectorizer(self):
        return self._vectorizer

    def set_split(self, split="train"):
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        row = self._target_df.iloc[index]
        text = str(row[self._text_col])
        x, y = self._vectorizer.vectorize(text, vector_length=self._max_seq_length)
        return {"x_data": torch.tensor(x, dtype=torch.long), "y_target": torch.tensor(y, dtype=torch.long)}

Empezamos el entrenamiento con el dataset de Hadith-s

In [71]:
def train_RNN():
    args = Namespace(
        hadith_csv="../data/hadith_dataset/hadith_en/hadith_en_cleaned.csv",
        vectorizer_file="vectorizer.json",
        model_state_file="model.pth",
        save_dir="Unai/Models/RNN/hadith/coran_rnn_v1",

        char_embedding_size=300, # 300 porque los embeddings del ft son de 300, tienen que coincidir
        rnn_hidden_size=128, # 256-ekin peatau itenzatek

        seed=1337,
        learning_rate=1e-3,
        batch_size=256,
        num_epochs=50,
        early_stopping_criteria=5,

        cuda=True,
        reload_from_files=False
    )

    print(args.batch_size)

    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.cuda and torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)
        args.device = torch.device("cuda")
    else:
        args.device = torch.device("cpu")

    os.makedirs(args.save_dir, exist_ok=True)
    if args.vectorizer_file and not os.path.isabs(args.vectorizer_file):
        args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)
    if args.model_state_file and not os.path.isabs(args.model_state_file):
        args.model_state_file = os.path.join(args.save_dir, args.model_state_file)

    if args.reload_from_files and os.path.exists(args.vectorizer_file):
        dataset = HadithDataset.load_dataset_and_load_vectorizer(args.hadith_csv, args.vectorizer_file, "text_en")
    else:
        dataset = HadithDataset.load_dataset_and_make_vectorizer(args.hadith_csv, "text_en")
        dataset.save_vectorizer(args.vectorizer_file)

    vectorizer = dataset.get_vectorizer()
    mask_index = vectorizer.char_vocab.mask_index

    def obtener_pesos(vectorizer, modelo_ft):
        vocab = vectorizer.char_vocab
        token_to_idx = vocab._token_to_idx
        tamaño_vocab = len(token_to_idx)
        embedding_dim = modelo_ft.get_dimension()
        pesos = np.zeros((tamaño_vocab, embedding_dim))

        for token, idx in token_to_idx.items():
            pesos[idx] = modelo_ft.get_word_vector(token)
    
        return torch.FloatTensor(pesos)

    ft_ingles = fasttext.load_model("../src/modelos/fasttext_english_busqueda_seamantica.bin")
    pretrained_ft_pesos = obtener_pesos(vectorizer, ft_ingles)

    model = CoranRNN(
        vocab_size=len(vectorizer.char_vocab),
        embedding_size=args.char_embedding_size,
        rnn_hidden_size=args.rnn_hidden_size,
        padding_idx=mask_index,
        pretrained_embeddings_ft=pretrained_ft_pesos
    ).to(args.device)

    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=1)

    train_state = make_train(args)

    for epoch in range(args.num_epochs):
        train_state["epoch_index"] = epoch

        # Train
        dataset.set_split("train")
        model.train()
        running_loss, running_acc = 0.0, 0.0
        for bi, batch in enumerate(generate_batches(dataset, args.batch_size, args.device, shuffle=True)):
            optimizer.zero_grad()
            y_pred = model(batch["x_data"])
            loss = sequence_loss(y_pred, batch["y_target"], mask_index)
            loss.backward()
            optimizer.step()

            running_loss += (loss.item() - running_loss) / (bi + 1)
            acc = compute_accuracy(y_pred, batch["y_target"], mask_index)
            running_acc += (acc - running_acc) / (bi + 1)

        train_state["train_loss"].append(running_loss)
        train_state["train_acc"].append(running_acc)

        # Val
        dataset.set_split("val")
        model.eval()
        vloss, vacc = 0.0, 0.0
        with torch.no_grad():
            for bi, batch in enumerate(generate_batches(dataset, args.batch_size, args.device, shuffle=False)):
                y_pred = model(batch["x_data"])
                loss = sequence_loss(y_pred, batch["y_target"], mask_index)

                vloss += (loss.item() - vloss) / (bi + 1)
                acc = compute_accuracy(y_pred, batch["y_target"], mask_index)
                vacc += (acc - vacc) / (bi + 1)

        train_state["val_loss"].append(vloss)
        train_state["val_acc"].append(vacc)

        train_state = update_training_state(args, model, train_state)
        scheduler.step(vloss)

        print(f"Epoch {epoch+1:03d} | train_loss={running_loss:.4f} "
              f"| val_loss={vloss:.4f} | val_acc={vacc:.4f}")
        
        dataset.save_vectorizer(args.vectorizer_file)
        torch.save(model.state_dict(), args.model_state_file)

        if train_state["stop_early"]:
            print("Early stopping activado.")
            break

    return args, dataset, vectorizer, model

In [72]:
args, dataset, vectorizer, model_rnn = train_RNN()

256


Epoch 001 | train_loss=2.8241 | val_loss=2.3093 | val_acc=0.3470
Epoch 002 | train_loss=2.2620 | val_loss=2.0432 | val_acc=0.4160
Epoch 003 | train_loss=2.0805 | val_loss=1.8840 | val_acc=0.4769
Epoch 004 | train_loss=1.9622 | val_loss=1.7745 | val_acc=0.4997
Epoch 005 | train_loss=1.8833 | val_loss=1.7060 | val_acc=0.5103
Epoch 006 | train_loss=1.8312 | val_loss=1.6579 | val_acc=0.5246
Epoch 007 | train_loss=1.7942 | val_loss=1.6244 | val_acc=0.5348
Epoch 008 | train_loss=1.7660 | val_loss=1.5998 | val_acc=0.5418
Epoch 009 | train_loss=1.7438 | val_loss=1.5799 | val_acc=0.5440
Epoch 010 | train_loss=1.7252 | val_loss=1.5609 | val_acc=0.5514
Epoch 011 | train_loss=1.7095 | val_loss=1.5480 | val_acc=0.5551
Epoch 012 | train_loss=1.6973 | val_loss=1.5351 | val_acc=0.5594
Epoch 013 | train_loss=1.6858 | val_loss=1.5284 | val_acc=0.5601
Epoch 014 | train_loss=1.6761 | val_loss=1.5202 | val_acc=0.5633
Epoch 015 | train_loss=1.6682 | val_loss=1.5119 | val_acc=0.5658
Epoch 016 | train_loss=1.

In [77]:
# number of verses to generate
num_names = 10

model = model_rnn.cpu()

sampled_verses = decode_samples(
    sample_from_model(
        model,
        vectorizer,
        num_samples=num_names,
        max_length=300,
        temperature=0.8
    ),
    vectorizer
)

# Show results
print("-" * 30)
for i in range(num_names):
    print(f"\n Verse {i+1}:\n{sampled_verses[i]}")

------------------------------

 Verse 1:
the propans and the same not of puraing with the is replaining tall o eros (lever and ont corneasting aponthto said i had the andar to the shares of that a sabdo chitode of the prophet ( ) seed fill in the versed comes theres) wall coured untin task bisk.

 Verse 2:
abamfaranm m and the for upen oneren)

 Verse 3:
abu huraira wat'lat thet was chatered, the informed and in me the messengor (whing the ponfited to the fristy in allah's mind betreced to him in the there of the prophet ( ) man and allah as the tereisers, i promelsand ibn iljat abna mat 'romins the mendared. bethe de in who perestle of nurdes of th

 Verse 4:
amal said (nasill and like nien say by allah's has the arearill belpen, in me and the fiss of allah, bedren josishen mots and the enters and him to whaling of not he said, ol al-sahima ha (messafiled an abrathe (of alllahab, b. al-muthal has not fitt the destle of al- am arrangither i comen. thereri

 Verse 5:
alia had a wah we 